# Table of Contents
1. [Imports](#imports)
2. [Import Data](#importData)
3. [Filter Data](#filter)
    1. [Arrivals](#filterArrivals)
         1. [Nieuwmarkt](#arrivalsNieuwmarkt)
         2. [Nieuwezijdse Kolk](#arrivalsNieuwezijds)
         3. [Dam](#arrivalsDam)
         4. [Spui](#arrivalsSpui)
         5. [Merge Arrivals](#arrivalsMerge)
    2. [Departures](#filterDepartures)
        1. [Nieuwmarkt](#departuresNieuwmarkt)
        2. [Nieuwezijdse Kolk](#departuresNieuwezijds)
        3. [Dam](#departuresDam)
        4. [Spui](#departuresSpui)
        5. [Merge Departures](#departuresMerge)
    3. [Merge Departues and Arrivals](#filterMerge)
4. [Construction DataFrame](#construction)
    1. [Data to Dict](#constructionDict)

# Imports <a name="imports"></a>

In [1]:
# For JSON imports
import json

#For DF, CSV, Excel
import pandas as pd

import re
import datetime

# Import the data <a name="importData"></a>

In [184]:
#Import dataset
arr_df = pd.read_csv("../../../Data_thesis/GVB/Datalab_Reis_Bestemming_Uur_20190402.csv", sep=";")
dep_df = pd.read_csv("../../../Data_thesis/GVB/Datalab_Reis_Herkomst_Uur_20190403.csv", sep=";")

#Rename 'AantalReizen' column
arr_df = arr_df.rename(index=str, columns={"AantalReizen": "NumberOfArrivals", "Datum": "Date",
                                           "UurgroepOmschrijving (van aankomst)": "Hour"})
dep_df = dep_df.rename(index=str, columns={"AantalReizen": "NumberOfDepartures", "Datum": "Date",
                                           "UurgroepOmschrijving (van vertrek)": "Hour"})

# Filter Data <a name="filter"></a>

In [185]:
#Selection of stations to use
stations = ["Nieuwmarkt", "Nieuwezijds Kolk", "Dam", "Spui"]

#Select rows that contain one of the listed stations in Aankomst
arr_df = arr_df[arr_df["AankomstHalteNaam"].isin(stations)]

#Select rows that contain one of the listed stations in Vertrek
dep_df = dep_df[dep_df["VertrekHalteNaam"].isin(stations)]

## Arrivals <a name="filterArrivals"></a>

In [186]:
arr_df.head()

,Date,Hour,AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,NumberOfArrivals
12,1/1/2018 12:00:00 AM,01:00 - 01:59,NMT,Nieuwmarkt,4.901239,52.371942,11
34,1/1/2018 12:00:00 AM,02:00 - 02:59,05055,Dam,4.890080,52.373127,21
68,1/1/2018 12:00:00 AM,02:00 - 02:59,NMT,Nieuwmarkt,4.901239,52.371942,48
92,1/1/2018 12:00:00 AM,03:00 - 03:59,05055,Dam,4.890080,52.373127,13
93,1/1/2018 12:00:00 AM,03:00 - 03:59,05069,Nieuwezijds Kolk,4.893731,52.376288,16


### Nieuwmarkt <a name="arrivalsNieuwmarkt"></a>

In [187]:
temp_df = arr_df[arr_df["AankomstHalteNaam"] == "Nieuwmarkt"]
nmt_codes = temp_df.AankomstHalteCode.unique()

arr_nmt_df = arr_df[arr_df["AankomstHalteCode"].isin(nmt_codes)]
arr_nmt_df = arr_nmt_df.rename(index=str, columns={"AankomstHalteCode": "NieuwmarktCode", "AankomstLat": "NieuwmarktLat", 
                                           "AankomstLon": "NieuwmarktLon", "NumberOfArrivals": "NieuwmarktArrivals"})
arr_nmt_df = arr_nmt_df.drop(columns=["AankomstHalteNaam"])

In [188]:
arr_nmt_df.head()

,Date,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals
12,1/1/2018 12:00:00 AM,01:00 - 01:59,NMT,4.901239,52.371942,11
68,1/1/2018 12:00:00 AM,02:00 - 02:59,NMT,4.901239,52.371942,48
118,1/1/2018 12:00:00 AM,03:00 - 03:59,NMT,4.901239,52.371942,10
153,1/1/2018 12:00:00 AM,04:00 - 04:59,NMT,4.901239,52.371942,16
192,1/1/2018 12:00:00 AM,05:00 - 05:59,NMT,4.901239,52.371942,17


### Nieuwezijdse Kolk <a name="arrivalsNieuwezijds"></a>

In [190]:
#All the station codes
temp_df = arr_df[arr_df["AankomstHalteNaam"] == "Nieuwezijds Kolk"]
nwk_codes = temp_df.AankomstHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
arr_nwk_df = arr_df[arr_df["AankomstHalteCode"].isin(nwk_codes)]

#Rename the columns
arr_nwk_df = arr_nwk_df.rename(index=str, columns={"AankomstHalteCode": "NieuwezijdsCode", "AankomstLat": "NieuwezijdsLat", 
                                           "AankomstLon": "NieuwezijdsLon", "NumberOfArrivals": "NieuwezijdsArrivals"})

#Drop the name column
arr_nwk_df = arr_nwk_df.drop(columns=["AankomstHalteNaam"])

#Combine the arrivlas of multiple codes at the same time and date
arr_nwk_df = arr_nwk_df.groupby(["Date", "Hour"]).agg({"NieuwezijdsCode": 'first',
                                               "NieuwezijdsLat": 'first',
                                               "NieuwezijdsLon": 'first',
                                               "NieuwezijdsArrivals": 'sum'}).reset_index()

### Dam <a name="arrivalsDam"></a>

In [191]:
#All the station codes
temp_df = arr_df[arr_df["AankomstHalteNaam"] == "Dam"]
dam_codes = temp_df.AankomstHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
arr_dam_df = arr_df[arr_df["AankomstHalteCode"].isin(dam_codes)]

#Rename the columns
arr_dam_df = arr_dam_df.rename(index=str, columns={"AankomstHalteCode": "DamCode", "AankomstLat": "DamLat", 
                                           "AankomstLon": "DamLon", "NumberOfArrivals": "DamArrivals"})

#Drop the name column
arr_dam_df = arr_dam_df.drop(columns=["AankomstHalteNaam"])

#Combine the arrivlas of multiple codes at the same time and date
arr_dam_df = arr_dam_df.groupby(["Date", "Hour"]).agg({"DamCode": 'first',
                                               "DamLat": 'first',
                                               "DamLon": 'first',
                                               "DamArrivals": 'sum'}).reset_index()

### Spui <a name="arrivalsSpui"></a>

In [192]:
#All the station codes
temp_df = arr_df[arr_df["AankomstHalteNaam"] == "Spui"]
spui_codes = temp_df.AankomstHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
arr_spui_df = arr_df[arr_df["AankomstHalteCode"].isin(spui_codes)]

#Rename the columns
arr_spui_df = arr_spui_df.rename(index=str, columns={"AankomstHalteCode": "SpuiCode", "AankomstLat": "SpuiLat", 
                                           "AankomstLon": "SpuiLon", "NumberOfArrivals": "SpuiArrivals"})

#Drop the name column
arr_spui_df = arr_spui_df.drop(columns=["AankomstHalteNaam"])

#Combine the arrivlas of multiple codes at the same time and date
arr_spui_df = arr_spui_df.groupby(["Date", "Hour"]).agg({"SpuiCode": 'first',
                                               "SpuiLat": 'first',
                                               "SpuiLon": 'first',
                                               "SpuiArrivals": 'sum'}).reset_index()

### Merge DataFrames <a name="arrivalsMerge"></a>

In [193]:
arrivals_1 = pd.merge(arr_nmt_df, arr_nwk_df, on=["Date", "Hour"], how="outer")
arrivals_2 = pd.merge(arr_dam_df, arr_spui_df, on=["Date", "Hour"], how="outer")
arrivals = pd.merge(arrivals_1, arrivals_2, on=["Date", "Hour"], how="outer")

In [194]:
arrivals.head()

,Date,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwezijdsCode,NieuwezijdsLat,NieuwezijdsLon,NieuwezijdsArrivals,DamCode,DamLat,DamLon,DamArrivals,SpuiCode,SpuiLat,SpuiLon,SpuiArrivals
0,1/1/2018 12:00:00 AM,01:00 - 01:59,NMT,4.901239,52.371942,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2018 12:00:00 AM,02:00 - 02:59,NMT,4.901239,52.371942,48.0,NaN,NaN,NaN,NaN,05055,4.89008,52.373127,21.0,NaN,NaN,NaN,NaN
2,1/1/2018 12:00:00 AM,03:00 - 03:59,NMT,4.901239,52.371942,10.0,05069,4.893731,52.376288,16.0,05055,4.89008,52.373127,13.0,NaN,NaN,NaN,NaN
3,1/1/2018 12:00:00 AM,04:00 - 04:59,NMT,4.901239,52.371942,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/2018 12:00:00 AM,05:00 - 05:59,NMT,4.901239,52.371942,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Departures <a name="filterDepartures"></a>

In [195]:
dep_df.head()

,Date,Hour,VertrekHalteCode,VertrekHalteNaam,VertrekLat,VertrekLon,NumberOfDepartures
6,1/1/2018 12:00:00 AM,01:00 - 01:59,05070,Nieuwezijds Kolk,4.892841,52.375754,27
18,1/1/2018 12:00:00 AM,01:00 - 01:59,NMT,Nieuwmarkt,4.901239,52.371942,340
29,1/1/2018 12:00:00 AM,02:00 - 02:59,05068,Dam,4.891241,52.374363,39
30,1/1/2018 12:00:00 AM,02:00 - 02:59,05070,Nieuwezijds Kolk,4.892841,52.375754,130
55,1/1/2018 12:00:00 AM,02:00 - 02:59,NMT,Nieuwmarkt,4.901239,52.371942,175


### Nieuwmarkt <a name="departuresNieuwmarkt"></a>

In [196]:
temp_df = dep_df[dep_df["VertrekHalteNaam"] == "Nieuwmarkt"]
nmt_codes = temp_df.VertrekHalteCode.unique()

dep_nmt_df = dep_df[dep_df["VertrekHalteCode"].isin(nmt_codes)]
dep_nmt_df = dep_nmt_df.rename(index=str, columns={"NumberOfDepartures": "NieuwmarktDepartures"})
dep_nmt_df = dep_nmt_df.drop(columns=["VertrekHalteNaam", "VertrekHalteCode", "VertrekLat", "VertrekLon"])

In [197]:
dep_nmt_df.head()

,Date,Hour,NieuwmarktDepartures
18,1/1/2018 12:00:00 AM,01:00 - 01:59,340
55,1/1/2018 12:00:00 AM,02:00 - 02:59,175
96,1/1/2018 12:00:00 AM,03:00 - 03:59,137
123,1/1/2018 12:00:00 AM,04:00 - 04:59,48
156,1/1/2018 12:00:00 AM,05:00 - 05:59,56


### Nieuwezijdse Kolk <a name="departuresNieuwezijds"></a>

In [198]:
#All the station codes
temp_df = dep_df[dep_df["VertrekHalteNaam"] == "Nieuwezijds Kolk"]
nwk_codes = temp_df.VertrekHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
dep_nwk_df = dep_df[dep_df["VertrekHalteCode"].isin(nwk_codes)]

#Rename the columns
dep_nwk_df = dep_nwk_df.rename(index=str, columns={"NumberOfDepartures": "NieuwezijdsDepartures"})

#Combine the arrivlas of multiple codes at the same time and date
dep_nwk_df = dep_nwk_df.groupby(["Date", "Hour"]).agg({"NieuwezijdsDepartures": 'sum'}).reset_index()

### Dam <a name="departuresDam"></a>

In [199]:
#All the station codes
temp_df = dep_df[dep_df["VertrekHalteNaam"] == "Dam"]
dam_codes = temp_df.VertrekHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
dep_dam_df = dep_df[dep_df["VertrekHalteCode"].isin(dam_codes)]

#Rename the columns
dep_dam_df = dep_dam_df.rename(index=str, columns={"NumberOfDepartures": "DamDepartures"})

#Combine the arrivlas of multiple codes at the same time and date
dep_dam_df = dep_dam_df.groupby(["Date", "Hour"]).agg({"DamDepartures": 'sum'}).reset_index()

### Spui <a name="departuresSpui"></a>

In [200]:
#All the station codes
temp_df = dep_df[dep_df["VertrekHalteNaam"] == "Spui"]
spui_codes = temp_df.VertrekHalteCode.unique()

#Make a subset of all the stations that have the given code(s)
dep_spui_df = dep_df[dep_df["VertrekHalteCode"].isin(spui_codes)]

#Rename the columns
dep_spui_df = dep_spui_df.rename(index=str, columns={"NumberOfDepartures": "SpuiDepartures"})

#Combine the arrivlas of multiple codes at the same time and date
dep_spui_df = dep_spui_df.groupby(["Date", "Hour"]).agg({"SpuiDepartures": 'sum'}).reset_index()

### Merge DataFrames <a name="departuresMerge"></a>

In [201]:
departures_1 = pd.merge(dep_nmt_df, dep_nwk_df, on=["Date", "Hour"], how="outer")
departures_2 = pd.merge(dep_dam_df, dep_spui_df, on=["Date", "Hour"], how="outer")
departures = pd.merge(departures_1, departures_2, on=["Date", "Hour"], how="outer")

In [202]:
departures.head()

,Date,Hour,NieuwmarktDepartures,NieuwezijdsDepartures,DamDepartures,SpuiDepartures
0,1/1/2018 12:00:00 AM,01:00 - 01:59,340.0,27.0,NaN,NaN
1,1/1/2018 12:00:00 AM,02:00 - 02:59,175.0,130.0,39.0,NaN
2,1/1/2018 12:00:00 AM,03:00 - 03:59,137.0,17.0,48.0,NaN
3,1/1/2018 12:00:00 AM,04:00 - 04:59,48.0,46.0,34.0,NaN
4,1/1/2018 12:00:00 AM,05:00 - 05:59,56.0,NaN,20.0,NaN


## Merge Arrivals and Departures <a name="filterMerge"></a>

In [216]:
full = pd.merge(arrivals, departures, on=["Date", "Hour"], how="outer")

In [217]:
#Reorder the columns
cols= ["Date", "Hour", "NieuwmarktCode", "NieuwmarktLat", "NieuwmarktLon", "NieuwmarktArrivals", 
            "NieuwmarktDepartures", "NieuwezijdsCode", "NieuwezijdsLat", "NieuwezijdsLon", "NieuwezijdsArrivals",
            "NieuwezijdsDepartures", "DamCode", "DamLat", "DamLon", "DamArrivals", "DamDepartures", "SpuiCode", "SpuiLat",
            "SpuiLon", "SpuiArrivals", "SpuiDepartures"]

full = full[cols]

#Replace the NaN value with 0
full = full.fillna(0.0)

#Add columns
#Gives day in numbers --> 0: Monday, 1: Tuesday,..., 6: Sunday
full["weekday"] = 99

#Adds whether day is weekend --> Day 5 and 6
full["is_weekend"] = 0

In [218]:
full.head()

,Date,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwmarktDepartures,NieuwezijdsCode,NieuwezijdsLat,NieuwezijdsLon,...,DamLon,DamArrivals,DamDepartures,SpuiCode,SpuiLat,SpuiLon,SpuiArrivals,SpuiDepartures,weekday,is_weekend
0,1/1/2018 12:00:00 AM,01:00 - 01:59,NMT,4.901239,52.371942,11.0,340.0,0,0.000000,0.000000,...,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,99,0
1,1/1/2018 12:00:00 AM,02:00 - 02:59,NMT,4.901239,52.371942,48.0,175.0,0,0.000000,0.000000,...,52.373127,21.0,39.0,0,0.0,0.0,0.0,0.0,99,0
2,1/1/2018 12:00:00 AM,03:00 - 03:59,NMT,4.901239,52.371942,10.0,137.0,05069,4.893731,52.376288,...,52.373127,13.0,48.0,0,0.0,0.0,0.0,0.0,99,0
3,1/1/2018 12:00:00 AM,04:00 - 04:59,NMT,4.901239,52.371942,16.0,48.0,0,0.000000,0.000000,...,0.000000,0.0,34.0,0,0.0,0.0,0.0,0.0,99,0
4,1/1/2018 12:00:00 AM,05:00 - 05:59,NMT,4.901239,52.371942,17.0,56.0,0,0.000000,0.000000,...,0.000000,0.0,20.0,0,0.0,0.0,0.0,0.0,99,0


# Construct Dataframes <a name="construction"></a>
Now that the contents is clear, we construct Dataframes for each station that is of interest to us. 

## Columns

**Arrival and Departure Stations:**
- *Metro*
    - Nieuwmarkt
- *Tram*
    - Nieuwezijdse Kolk
    - Dam
    - Spui
- *Type*: String

**Weekday:** Select the day number of the week, based on the given date
- *Type*: Int

**Weekend:** 1 if day is a weekend day
- *Type*: Binary

## Data to Dict <a name="constructionDict"></a>
Needed to ease certain operations

In [219]:
full_dict = full.to_dict("index")

In [220]:
#Datetime format
date_format_1 = '%d/%m/%Y %H:%M:%S'
date_format_2 = '%m/%d/%Y %H:%M:%S'

#Loop over dict
for k, v in full_dict.items():
    #Replace time string with time blok
    time_blok = v["Hour"][:5]
    time_blok = re.sub('[:]', '', time_blok)
    v["Hour"] = int(time_blok)
    
    try:
        #Remove AM/PM from string
        v["Date"] = v["Date"][:-3]
        try:
            #Transform the date string to datatime.date object
            date = datetime.datetime.strptime(v["Date"], date_format_1)
            #Transfrom date to weekday number
            v["weekday"] = date.weekday()
        except:
            #Transform the date string to datatime.date object
            date = datetime.datetime.strptime(v["Date"], date_format_2)
            
            #Transfrom date to weekday number
            v["weekday"] = date.weekday()
        v["Date"] = date.date()
    except:
          v["weekday"] = 99  
    
    #Check if weekday is in the weekend
    if date.weekday() == 5 or date.weekday() == 6:
        v["is_weekend"] = 1
        
    v["Date"] = date.date()

In [221]:
#Return to dict
full = pd.DataFrame.from_dict(full_dict, orient="index")

## Data Preperation
Make the model with the GVB data from *Dam* station

Variables final df:
- *UurgroepOmschrijving (van vertrek)*
    - The hour the counts were gathered --> 100 means 01:00
    - int
- *VertrekHalteNaam* / *AankomstHalteNaam*
    - Name of the leaving/arrival station
    - str
- *VertrekLat* / *AankomstLat*
    - Latitude leaving/arrival station
    - int
- *VertrekLon* / *AankomstLon*
    - Longitude leaving/arrival station
    - int
- *weekday*
    - Day of the week in numbers --> 0: Monday, 1: Tuesday,..., 6: Sunday
    - int
- *is_weekend*
    - Whether is it is weekend or not
    - binary
- *AantalRitten*
    - Number of the rides takes from the station
    - int

In [222]:
full.head()

,Date,Hour,NieuwmarktCode,NieuwmarktLat,NieuwmarktLon,NieuwmarktArrivals,NieuwmarktDepartures,NieuwezijdsCode,NieuwezijdsLat,NieuwezijdsLon,...,DamLon,DamArrivals,DamDepartures,SpuiCode,SpuiLat,SpuiLon,SpuiArrivals,SpuiDepartures,weekday,is_weekend
0,2018-01-01,100,NMT,4.901239,52.371942,11.0,340.0,0,0.000000,0.000000,...,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
1,2018-01-01,200,NMT,4.901239,52.371942,48.0,175.0,0,0.000000,0.000000,...,52.373127,21.0,39.0,0,0.0,0.0,0.0,0.0,0,0
2,2018-01-01,300,NMT,4.901239,52.371942,10.0,137.0,05069,4.893731,52.376288,...,52.373127,13.0,48.0,0,0.0,0.0,0.0,0.0,0,0
3,2018-01-01,400,NMT,4.901239,52.371942,16.0,48.0,0,0.000000,0.000000,...,0.000000,0.0,34.0,0,0.0,0.0,0.0,0.0,0,0
4,2018-01-01,500,NMT,4.901239,52.371942,17.0,56.0,0,0.000000,0.000000,...,0.000000,0.0,20.0,0,0.0,0.0,0.0,0.0,0,0


## DF to File
Save the dataframe in a file, so that it can be imported for other uses. Later this will be probably be removed as the dataframe can simple function as input to another funtion. 

In [223]:
full.to_csv("../../../Data_thesis/Full_Datasets/GVB.csv", index=False)